In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
locations = 'Lào Cai, Yên Bái, Điện Biên, Hòa Bình, Lai Châu, Sơn La, Hà Giang, Cao Bằng, Bắc Kạn, Lạng Sơn, Tuyên Quang, Thái Nguyên, Phú Thọ, Bắc Giang, Quảng Ninh, Bắc Ninh, Hà Nam, Hà Nội, Hải Dương, Hưng Yên, Hải Phòng, Nam Định, Ninh Bình, Thái Bình, Vĩnh Phúc, Thanh Hoá , Nghệ An'

In [4]:
import re
import sys

patterns = {
    '[àáảãạăắằẵặẳâầấậẫẩ]': 'a',
    '[đ]': 'd',
    '[èéẻẽẹêềếểễệ]': 'e',
    '[ìíỉĩị]': 'i',
    '[òóỏõọôồốổỗộơờớởỡợ]': 'o',
    '[ùúủũụưừứửữự]': 'u',
    '[ỳýỷỹỵ]': 'y'
}

def convert(text):
    """
    Convert from 'Tieng Viet co dau' thanh 'Tieng Viet khong dau'
    text: input string to be converted
    Return: string converted
    """
    output = text
    for regex, replace in patterns.items():
        output = re.sub(regex, replace, output)
        # deal with upper case
        output = re.sub(regex.upper(), replace.upper(), output)
    return output

In [5]:
# citys = convert(locations)
# citys = [i.lower().strip() for i in citys.split(", ")]
# citys=['/' + i.replace(" ",'-') for i in citys]
# citys2ids = requests.get("https://thongtindoanhnghiep.co/api/city").json()['LtsItem']
# citys2ids = [i for i in citys2ids if i['SolrID'] in citys]

In [6]:
# districts = []
# for city in citys2ids:
#     city_id = city['ID']
#     url = f'https://thongtindoanhnghiep.co/api/city/{city_id}/district'
#     r = requests.get(url)
#     districts += r.json()
    
# districts = [i['SolrID'][1:] for i in districts]

In [7]:
# num_records_by_districts = {}
# e = 0
# for i in districts:
#     e+=1
#     if(e%20) == 0:
#         print(e)
#     url = f'https://thongtindoanhnghiep.co/api/company?l={i}'
#     r = requests.get(url).json()
#     num_records_by_districts[i] = r["Option"]['TotalRow']
    

In [8]:
import pickle

# with open('num_records_by_districts.pickle', 'wb') as handle:
#     pickle.dump(num_records_by_districts, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [9]:

with open('num_records_by_districts.pickle', 'rb') as handle:
    num_records_by_districts = pickle.load(handle)



In [10]:
def get_closed_date(tax_code):
    url = f'https://thongtindoanhnghiep.co/api/company/{tax_code}'
    r = requests.get(url).json()
    closed_date = r['NgayDongMST']
    return closed_date


In [11]:
import random
import time
import pandas as pd

In [12]:
def parse_districts_html(district):
    num_records = num_records_by_districts[district]
    num_pages = num_records//15 + 2
    for page in range(1,num_pages):
        time.sleep(0.1)
        url = f'https://thongtindoanhnghiep.co/{district}?p={page}'
        r = requests.get(url)
        soup = BeautifulSoup(r.content)
        firms = soup.findAll("div", attrs={'class':'news-v3 bg-color-white'})
        res = [{'company_name': firm.find("h2").text, 
                  'location_detail': firm.findAll("strong")[-2].text,
                  'tax_code': firm.findAll("strong")[0].text,
                  'established_date': firm.findAll('p')[1].text.split(" ")[-1],
                  'sector': firm.findAll("strong")[-1].text,
                  'province': firm.findAll("strong")[1].text
                 } for firm in firms]
        df = pd.DataFrame(res, columns=['company_name','province','location_detail', 'sector', 'tax_code', 'established_date'])
        fn = district.split("/")[0] + '.csv'
        with open('./crawled/'+fn, 'a') as f:
            df.to_csv(f, index = False, header = False, sep='\t')
    print('DONE', district, num_records)

In [13]:
def parse_districts_api(district):
    num_records = num_records_by_districts[district]
    sleep = random.randint(1,10)
    time.sleep(sleep)
    num_pages = num_records//100 + 2
    for page in range(1,num_pages):
        time.sleep(0.1)
        url = f'https://thongtindoanhnghiep.co/api/company?r=100&p={page}&l={district}'
        try:
            r = requests.get(url).json()['LtsItems']
            res = [{'company_name': i['Title'], 
                      'location_detail': i['DiaChiCongTy'],
                      'tax_code': i['MaSoThue'],
                      'established_date': i['NgayCap'],
                      'sector': i['NganhNgheTitle'],
                      'province': i['TinhThanhTitle']
                     } for i in r]
            df = pd.DataFrame(res, columns=['company_name','province','location_detail', 'sector', 'tax_code', 'established_date'])
            fn = district.split("/")[0] + '.csv'
            with open('./crawled/' + fn, 'a') as f:
                df.to_csv(f, index = False, header = False, sep='\t')
        except:
            with open("./error.txt", 'a+') as f:
                print('error', url)

                time.sleep(120)
                f.write(f'{url}\n')
    print('DONE', district, num_records)

TypeError: string indices must be integers

In [32]:
ward = wards[0]

In [33]:
url = f'https://thongtindoanhnghiep.co/{ward}'

In [34]:
url

'https://thongtindoanhnghiep.co/ha-noi/huyen-dong-anh/khu-cong-nghiep-thang-long-1'

In [35]:
r = requests.get(url)

In [36]:
soup = BeautifulSoup(r.content)

'1'

In [43]:
soup

<!DOCTYPE html>
<!--[if IE 8]> <html lang="en" class="ie8"> <![endif]--><!--[if IE 9]> <html lang="en" class="ie9"> <![endif]--><!--[if !IE]><!--><html lang="en">
<!--<![endif]-->
<head>
<title>Danh sách doanh nghiệp tại Khu Công Nghiệp Thăng Long 1 - Huyện Đông Anh - Hà Nội</title>
<!-- Meta -->
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="thongtindoanhnghiep.co" name="author"/>
<meta content="5C8CDF0992489498A30F9E5F6668A4D5" name="msvalidate.01"/>
<meta content="6b2d2802dd6f6ba6" name="yandex-verification"/>
<link href="/rss/ha-noi/huyen-dong-anh/khu-cong-nghiep-thang-long-1" rel="alternate" title="ThongTinDoanhNghiep.Co: Danh sách doanh nghiệp tại Khu Công Nghiệp Thăng Long 1 - Huyện Đông Anh - Hà Nội RSS Feed" type="application/rss+xml"/>
<meta content="Danh sách doanh nghiệp tại Khu Công Nghiệp Thăng Long 1 - Huyện Đông Anh - Hà Nội" property="article:section"/>
<meta content="Xem, tra cứu doanh nghiệp tại Khu Công

In [26]:
def parse_wards_html(district):
    url = f'https://thongtindoanhnghiep.co/{district}'
    r = requests.get(url).content
    soup = BeautifulSoup(r)
    num_records = int(soup.find('div', attrs= {'class':"alert alert-warning fade in margin-bottom-40"}).find("strong").text.replace(',',''))
    num_pages = num_records//15 + 2
    for page in range(1,num_pages):
        time.sleep(0.1)
        url = f'https://thongtindoanhnghiep.co/{district}?p={page}'
        r = requests.get(url)
        soup = BeautifulSoup(r.content)
        firms = soup.findAll("div", attrs={'class':'news-v3 bg-color-white'})
        res = [{'company_name': firm.find("h2").text, 
                  'location_detail': firm.findAll("strong")[-2].text,
                  'tax_code': firm.findAll("strong")[0].text,
                  'established_date': firm.findAll('p')[1].text.split(" ")[-1],
                  'sector': firm.findAll("strong")[-1].text,
                  'province': firm.findAll("strong")[1].text
                 } for firm in firms]
        df = pd.DataFrame(res, columns=['company_name','province','location_detail', 'sector', 'tax_code', 'established_date'])
        fn = district.split("/")[0] + '.csv'
        with open('./crawled/'+fn, 'a') as f:
            df.to_csv(f, index = False, header = False, sep='\t')
    print('DONE', district, num_records)

In [22]:
wards = [i['SolrID'] for i in wards]
wards = [i[1:] for i in wards]

TypeError: string indices must be integers

In [27]:
from multiprocessing import Pool
p = Pool(7)

DONE ha-noi/quan-ba-dinh/phuong-thanh-cong-bac 1
DONE ha-noi/quan-hai-ba-trung/phuong-mk 1
DONE ha-noi/quan-hai-ba-trung/phuong-o-cau-den 7
DONE ha-noi/quan-cau-giay/phuong-d-vong 2
DONE ha-noi/huyen-dong-anh/khu-cong-nghiep-thang-long-1 1
DONE ha-noi/quan-bac-tu-liem/xa-thuy-phuong 4
DONE ha-noi/huyen-ba-vi/xa-phu-phuong 15
DONE ha-noi/quan-bac-tu-liem/xa-tay-tuu 3
DONE ha-noi/quan-hai-ba-trung/xa-viet-nam 1
DONE ha-noi/quan-long-bien/thi-tran-93-duc-giang 1
DONE ha-noi/quan-long-bien/khu-cong-nghiep-sai-dong-b 12
DONE ha-noi/quan-long-bien/khu-cong-nghiep-ky-thuat-cao-sai-dong-b 1
DONE ha-noi/huyen-ba-vi/xa-van-thang 45
DONE ha-noi/quan-hai-ba-trung/phuong-ngo-thi-nham 617
DONE ha-noi/quan-hai-ba-trung/phuong-nguyendu 1
DONE ha-noi/quan-long-bien/phuong-thach-ban 615
DONE ha-noi/quan-long-bien/xa-thach-ban 1
DONE ha-noi/quan-ba-dinh/phuong-truc-bach 905
DONE ha-noi/huyen-dong-anh/thi-tran-dong-anh 1155
DONE ha-noi/huyen-dong-anh/khu-cong-nghiep-dong-anh 1
DONE ha-noi/huyen-dong-anh/t

DONE ha-noi/huyen-chuong-my/xa-phu-nghia 175
DONE ha-noi/huyen-chuong-my/xa-ngoc-son 6
DONE ha-noi/quan-hai-ba-trung/phuong-dong-mac 428
DONE ha-noi/huyen-chuong-my/xa-tran-phu 76
DONE ha-noi/quan-ba-dinh/chua-ro 4519
DONE ha-noi/huyen-chuong-my/xa-tien-phuong 121
DONE ha-noi/huyen-chuong-my/thi-tran-truc-son 2
DONE ha-noi/huyen-chuong-my/xa-tan-tien 42
DONE ha-noi/huyen-chuong-my/xa-thuong-vuc 17
DONE ha-noi/huyen-chuong-my/xa-thuy-xuan-tien 118
DONE ha-noi/huyen-chuong-my/xa-truong-yen 156
DONE ha-noi/quan-long-bien/phuong-duc-giang 1353
DONE ha-noi/huyen-chuong-my/xa-van-vo 12
DONE ha-noi/quan-long-bien/thi-tran-duc-giang 11
DONE ha-noi/quan-bac-tu-liem/phuong-xuan-dinh 1310
DONE ha-noi/huyen-chuong-my/xa 1
DONE ha-noi/quan-bac-tu-liem/xa-xuan-dinh 15
DONE ha-noi/quan-dong-da/xa-dan-2 18
DONE ha-noi/quan-cau-giay/phuong-mai-dich 2063
DONE ha-noi/quan-dong-da/phuong-giap-bat 1
DONE ha-noi/quan-cau-giay/phuongmai-dich 1
DONE ha-noi/quan-cau-giay/phuong-dich-v-hau 1
DONE ha-noi/quan-do

DONE ha-noi/huyen-hoai-duc/thi-tran 1
DONE ha-noi/huyen-my-duc/xa-le-thanh 17
DONE ha-noi/huyen-hoai-duc/xa-tien-yen 35
DONE ha-noi/huyen-hoai-duc/xa-van-xuan 1
DONE ha-noi/huyen-hoai-duc/xa 2
DONE ha-noi/huyen-my-duc/xa-phu-luu-te 43
DONE ha-noi/huyen-my-duc/xa-an-tien 20
DONE ha-noi/huyen-my-duc/xa-my-thanh 11
DONE ha-noi/huyen-my-duc/xa-an-phu 18
DONE ha-noi/quan-ha-dong/phuong-phu-la 1168
DONE ha-noi/huyen-my-duc/xa-an-my 33
DONE ha-noi/huyen-my-duc/xa-phung-xa 75
DONE ha-noi/huyen-my-duc/xa-bot-xuyen 13
DONE ha-noi/huyen-my-duc/thi-tran-te-tieu 1
DONE ha-noi/huyen-my-duc/xa-xuy-xa 23
DONE ha-noi/huyen-gia-lam/phuong-109-to-nong-lam 1
DONE ha-noi/huyen-gia-lam/xa-ai-mo 1
DONE ha-noi/quan-dong-da/chua-ro 4961
DONE ha-noi/huyen-gia-lam/khu-cong-nghiep-sai-dong-b 2
DONE ha-noi/huyen-gia-lam/xa-ba-trang 1
DONE ha-noi/quan-ba-dinh/phuong-nguyen-trung-truc 442
DONE ha-noi/quan-ba-dinh/phuong-yen-ninh 1
DONE ha-noi/huyen-my-duc/chua-ro 163
DONE ha-noi/huyen-gia-lam/xa-thach-ban 2
DONE ha-

DONE ha-noi/quan-ha-dong/xa-van-phuc 9
DONE ha-noi/thanh-pho-ha-noi/chua-ro 378
DONE ha-noi/huyen-thanh-tri/xa-tam-hiep 528
DONE ha-noi/thanh-pho-ha-noi/khu-cong-nghiep-thang-long 15
DONE ha-noi/thanh-pho-ha-noi/phuong-thanh-cong 1
DONE ha-noi/thanh-pho-ha-noi/khu-cong-nghiep-quang-minh 1
DONE ha-noi/thanh-pho-ha-noi/khu-cong-nghiep-quang-minh-mo-rong 1
DONE ha-noi/thanh-pho-ha-noi/phuong-cong-vi 2
DONE ha-noi/huyen-thuong-tin/xa-hien-giang 19
DONE ha-noi/quan-hoan-kiem/phuong-hang-buom 500
DONE ha-noi/quan-hoan-kiem/phuong-phan-chi-trinh 1
DONE ha-noi/huyen-thuong-tin/xa-khanh-ha 75
DONE ha-noi/quan-hoan-kiem/phuong-phan-chu-chinh 2
DONE ha-noi/quan-hoan-kiem/phuong-tran-hung-dao 1069
DONE ha-noi/huyen-thuong-tin/xa-ha-hoi 71
DONE ha-noi/huyen-thuong-tin/xa-to-hieu 72
DONE ha-noi/huyen-thanh-tri/xa-ngu-hiep 385
DONE ha-noi/quan-hoan-kiem/phuong-hang-dao 165
DONE ha-noi/quan-hoan-kiem/phuong-tran-h-dao 3
DONE ha-noi/quan-hoan-kiem/phuong-ly-nam-de 1
DONE ha-noi/huyen-thuong-tin/xa-hong

DONE ha-noi/huyen-thanh-oai/xa-tam-hung 79
DONE ha-noi/huyen-thanh-oai/xa-kim-thu 25
DONE ha-noi/huyen-soc-son/xa-duc-hoa 33
DONE ha-noi/huyen-thanh-oai/xa-phu-lam 2
DONE ha-noi/huyen-thanh-oai/xa-phu-luong 3
DONE ha-noi/huyen-me-linh/xa-dai-thinh 171
DONE ha-noi/huyen-thanh-oai/xa-thanh-mai 37
DONE ha-noi/huyen-me-linh/xa-tam-dong 42
DONE ha-noi/huyen-me-linh/xa-gia-trung 1
DONE ha-noi/huyen-thanh-oai/xa-phuong-trung 75
DONE ha-noi/huyen-soc-son/xa-tien-duoc 153
DONE ha-noi/huyen-me-linh/xa-kim-hoa 64
DONE ha-noi/huyen-thanh-oai/xa-tan-uoc 22
DONE ha-noi/huyen-me-linh/xa-hoang-kim 41
DONE ha-noi/quan-tay-ho/phuong-yen-phu 1145
DONE ha-noi/huyen-soc-son/xa-trung-gia 155
DONE ha-noi/huyen-me-linh/xa-van-khe 86
DONE ha-noi/huyen-thuong-tin/chua-ro 397
DONE ha-noi/huyen-thanh-oai/xa-thanh-thuy 132
DONE ha-noi/huyen-soc-son/xa-xuan-giang 37
DONE ha-noi/huyen-thuong-tin/xa-chuong-duong 28
DONE ha-noi/huyen-thanh-oai/xa-thanh-van 34
DONE ha-noi/huyen-thuong-tin/cum-cong-nghiep-quat-dong 3
DO

DONE ha-noi/quan-hoang-mai/chua-ro 3421
DONE ha-noi/huyen-tu-liem/khu-cong-nghiep-tu-liem 1
DONE ha-noi/quan-hoang-mai/xa-dinh-co 1
DONE ha-noi/huyen-tu-liem/cum-cong-nghiep-vua-va-nho-tu-liem 1
DONE ha-noi/huyen-tu-liem/xa-dai-mo 18
DONE ha-noi/huyen-thanh-oai/chua-ro 246
DONE ha-noi/huyen-tu-liem/xa-phu-dien 20
DONE ha-noi/huyen-thanh-oai/cum-cong-nghiep-thanh-oai 5
DONE ha-noi/huyen-thanh-oai/cum-cong-nghiep 1
DONE ha-noi/thi-xa-son-tay/chua-ro 313
DONE ha-noi/huyen-tu-liem/xa-my-dinh 123
DONE ha-noi/huyen-tu-liem/xa-xuan-dinh 23
DONE ha-noi/huyen-tu-liem/xa-my-dinh-i 1
DONE ha-noi/huyen-tu-liem/phuong-nghia-do 1
DONE ha-noi/huyen-tu-liem/xa-dong-ngac 12
DONE ha-noi/thi-xa-son-tay/xa-co-dong 99
DONE ha-noi/huyen-tu-liem/xa-minh-khai 17
DONE ha-noi/huyen-tu-liem/xa-lien-mac 7
DONE ha-noi/thi-xa-son-tay/xa-son-dong 62
DONE ha-noi/quan-nam-tu-liem/phuong-my-dinh-1 1991
DONE ha-noi/thi-xa-son-tay/phuong-son-dong 1
DONE ha-noi/quan-hoang-mai/phuong-yen-so 632
DONE ha-noi/thi-xa-son-tay/x

In [ ]:
p.map(parse_wards_html, wards)

In [16]:
districts = num_records_by_districts.keys()

In [16]:
# p.map(parse_districts, [i for i in districts if num_records_by_districts[i] < 10000])

In [17]:
remaining = [i for i in districts if i.startswith('ha-noi')]

In [14]:
district_ids = requests.get("https://thongtindoanhnghiep.co/api/city/3/district").json()

In [15]:
district_ids = [i['ID'] for i in district_ids]

In [16]:
len(district_ids)

32

In [17]:
wards = []
for district_id in district_ids:
    ward = requests.get(f"https://thongtindoanhnghiep.co/api/district/{district_id}/ward").json()
    wards += ward

In [18]:
len(wards)

1101

In [18]:
p.map(parse_districts_api, remaining)

KeyboardInterrupt: 

In [22]:
num_records_by_districts

{'hung-yen/huyen-an-thi': 493,
 'hung-yen/huyen-khoai-chau': 1241,
 'hung-yen/huyen-phu-cu': 338,
 'hung-yen/huyen-kim-dong': 594,
 'hung-yen/huyen-van-giang': 980,
 'hung-yen/huyen-my-hao': 1669,
 'hung-yen/thi-xa-my-hao': 6,
 'hung-yen/thanh-pho-hung-yen': 1663,
 'hung-yen/tinh-hung-yen': 26,
 'hung-yen/huyen-yen-my': 1894,
 'hung-yen/huyen-van-lam': 2269,
 'hung-yen/huyen-tien-lu': 451,
 'ha-noi/huyen-dong-anh': 6588,
 'ha-noi/quan-ba-dinh': 20904,
 'ha-noi/quan-hai-ba-trung': 20745,
 'ha-noi/huyen-ba-vi': 1201,
 'ha-noi/quan-bac-tu-liem': 9821,
 'ha-noi/quan-long-bien': 12230,
 'ha-noi/quan-cau-giay': 27307,
 'ha-noi/huyen-chuong-my': 3211,
 'ha-noi/quan-dong-da': 30209,
 'ha-noi/huyen-dan-phuong': 2162,
 'ha-noi/quan-ha-dong': 15642,
 'ha-noi/huyen-hoai-duc': 4095,
 'ha-noi/huyen-my-duc': 929,
 'ha-noi/huyen-gia-lam': 4932,
 'ha-noi/thanh-pho-ha-noi': 441,
 'ha-noi/quan-tay-ho': 9743,
 'ha-noi/huyen-ung-hoa': 951,
 'ha-noi/quan-hoan-kiem': 14209,
 'ha-noi/quan-hoang-mai': 21824,
 